In [1]:
import numpy as np
import sys
import torch
import pandas as pd
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as Data
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import jieba
import emoji
from gensim.models import Word2Vec, KeyedVectors

import Model

In [2]:
x_path = 'train_x.csv'
y_path = 'train_y.csv'
dict_txt_path = 'dict.txt.big'
x_type = {'id': int, 'comment':str}
y_type = {'id': int, 'label':int}
vecSize = 200

In [3]:
jieba.load_userdict(dict_txt_path)
x = pd.read_csv(x_path, sep=',', dtype=x_type, index_col=0)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.478 seconds.
Prefix dict has been built succesfully.


In [ ]:
wv = KeyedVectors.load("word2vec.model", mmap='r')
x_train = np.zeros(shape=(len(x), 64, vecSize), dtype=float)
for i_row, sen in enumerate(x['comment']):
    for i_w, w in enumerate(list(jieba.cut(emoji.demojize(sen), cut_all=False))):
        if i_w >= 64: break
        x_train[i_row, i_w, :] = wv[w]
#x_array = [ wv[word] for word in jieba.cut(emoji.demojize(sentence), cut_all=False) for sentence in x['comment'] ]
#max_s_len = x['comment'].map(lambda x: len(x)).max()
#x_len = len(x)
#idx =x['comment'].map(lambda x: len(x)).idxmax()

In [6]:
y = pd.read_csv(y_path, sep=',', dtype=y_type, index_col=0)
y_train = y['label'].values


In [7]:
EPOCH = 5
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [8]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()

        self.rnn = nn.LSTM(
            input_size=200,
            hidden_size=64,
            num_layers=1,
            batch_first=True,
            #dropout=0.3
        )
        self.fc = nn.Linear(64, 1)
        self.out = nn.Sigmoid()
    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)   LSTM 有两个 hidden states, h_n 是分线, h_c 是主线
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None 表示 hidden state 会用全0的 state

        # 选取最后一个时间点的 r_out 输出
        # 这里 r_out[:, -1, :] 的值也是 h_n 的值
        fc = self.fc(r_out[:, -1, :])
        out = self.out(fc)
        return out

model = LSTM()
device = torch.device('cuda')
model.to(device)
model.train()
print(model)

RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED

In [ ]:
dataset = Data.TensorDataset(
        torch.tensor(x_train).type(torch.FloatTensor), 
        torch.tensor(y_train).type(torch.LongTensor))
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_func = nn.CrossEntropyLoss() 

print('====== Start Training ======')
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        x_cuda = x.to(device, dtype=torch.float)
        y_cuda = y.to(device)
        print(x.Size())
        x = x.view(-1, 64, 200)   # reshape x to (batch, time_step, input_size)

        output = model(b_x)               # rnn output
        loss = loss_func(output, y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
torch.save(model.state_dict(), 'model_params.pkl') # parameters